Life cycle of a Data science project
1. Data collection
2. Feature Engineering
3. Feature Selection
4. Model creation
5. Hyper parameter Tuning

 All techniques of Hyper parameter Optimization
 
    1. GridsearchCV
    2. RandamizedSearchCV
    3. Bayesian Optimization - Automate Hyperparameter Tuning(Hyperopt)
    4. Sequential Model based optimization(Tuning scikit-learn estimator with skopt)
    5. Optuna-Automate Hyperparameter Tuning
    6. Genetic Algorithms (TPOT Classifier) -- Much efficient on Huge data.

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
df=pd.read_csv('Datasets\diabetes.csv')
df.head()

c:\users\cdileepkumar\documents\02 python_practice\03 softwares\01 python\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\cdileepkumar\documents\02 python_practice\03 softwares\01 python\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [1]:
#Scaling is not required in Random forest algorithms,bcoz things work on decision tress 
#and no much mathematical formulas will get applied on data.

In [6]:
##Indepedent and depenedent Feature selection
x=df.drop(["Outcome"],axis=1)
y=df['Outcome']
x.head()
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

In [9]:
## Check zero values in columns, which are not supposed to be
import numpy as np
df['Glucose']=np.where(df['Glucose'] == 0,df['Glucose'].mean(),df['Glucose'])
df['BloodPressure']=np.where(df['BloodPressure'] == 0,df['BloodPressure'].mean(),df['BloodPressure'])
df['Insulin']=np.where(df['Insulin'] == 0,df['Insulin'].mean(),df['Insulin'])

In [11]:
#Train Test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=100)
x_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
660,10,162,84,0,0,27.7,0.182,54
69,4,146,85,27,100,28.9,0.189,27
85,2,110,74,29,125,32.4,0.698,27
219,5,112,66,0,0,37.8,0.261,41
712,10,129,62,36,0,41.2,0.441,38


In [14]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=10).fit(x_train,y_train)
y_pred=rf.predict(x_test)
y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [19]:
y_pred=pd.Series(y_pred)
y_pred.head()

0    0
1    0
2    1
3    0
4    0
dtype: int64

In [21]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.74      0.82      0.78       101
           1       0.57      0.45      0.51        53

    accuracy                           0.69       154
   macro avg       0.66      0.64      0.64       154
weighted avg       0.68      0.69      0.69       154

0.6948051948051948
[[83 18]
 [29 24]]


#The main parameters used by a Randomforest classifier

Criterion -> The function used to evaluate the quality of a split

max_depth -> maximum number of trees allowed in each tree

max_features -> maximum number of features consideered when splitting a node

min_sample_leaf -> minimum no.of samples which can be stored in a tree leaf

min_sample_split -> minimum no.of samples necessary in a node to cause node splitting.

n_estimators -> no.of trees in the ensamble.


In [28]:
## Manual Hyperparameter Tuning
rf=RandomForestClassifier(n_estimators=300,criterion='entropy',max_features='sqrt',min_samples_leaf=10,random_state=100).fit(x_train,y_train)
y_pred=rf.predict(x_test)
pd.Series(y_pred).value_counts()
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.76      0.83      0.80       101
           1       0.61      0.51      0.56        53

    accuracy                           0.72       154
   macro avg       0.69      0.67      0.68       154
weighted avg       0.71      0.72      0.71       154

0.7207792207792207
[[84 17]
 [26 27]]


#GridsearchCV vs Randomized searchCV
1 million people living in Area A,B,C,D

Find a person krish Naik in which area he is?

Gridsearchcv -> search all areas A,B,C,D-> finalises that he lives in area B.

-> There is no way to mention interations in gridsearchcv. It will calculate dynamically based on parameters.

Takes all the combinations and permutations..

Randomized searchCV -> Search some areas in less time and gives -> B,C -> give it to gridsearchcv -> it gives B

Do some less iterations and get what you get ..

Randomized searchCV -> Narrow down our results.

# Randomized Search CV

In [42]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
#Number of Trees in Randome Forest
n_estimators=[int(x) for x in np.linspace(start=200,stop=2000,num=10)]
n_estimators

[200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]

In [43]:
#Number of features to consider at every split
max_features=['auto','sqrt','log2']
#Maximum no.of levels in tree
max_depth=[int(x) for x in np.linspace(10,1000,10)]
#minimum no.of samples required to split a node
min_samples_split=[2,5,10,14]
#minimum no.of samples required at each leaf node
min_samples_leaf=[1,2,4,6,8]
#create the random gri
random_grid={'n_estimators':n_estimators,'max_features':max_features,'max_depth':max_depth,'min_samples_split':min_samples_split,
            'min_samples_leaf':min_samples_leaf,'criterion':['entropy','gini']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [ ]:
rf=RandomForestClassifier()
rf_randomscv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=100,cv=3,random_state=100,n_jobs=-1)
rf_randomscv.fit(x_train,y_train)

rf_randomscv.best_params_
rf_randomscv.best_estimator_

# This takes more time to execute

#Important Paraments 
#n_iter = 100 -> No.of iterations
#cv = 3 -> cross validation =3 means 3 times different samples will be taken from train data.
#no.of fits = 100 *3 = 300

In [ ]:
best_random_grid=rf_randomscv.best_estimator_

y_pred=best_random_grid.predict(x_test)

print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

# GridSearch CV

In [44]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': [rf_randomcv.best_params_['criterion']],
    'max_depth': [rf_randomcv.best_params_['max_depth']],
    'max_features': [rf_randomcv.best_params_['max_features']],
    'min_samples_leaf': [rf_randomcv.best_params_['min_samples_leaf'], 
                         rf_randomcv.best_params_['min_samples_leaf']+2, 
                         rf_randomcv.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rf_randomcv.best_params_['min_samples_split'] - 2,
                          rf_randomcv.best_params_['min_samples_split'] - 1,
                          rf_randomcv.best_params_['min_samples_split'], 
                          rf_randomcv.best_params_['min_samples_split'] +1,
                          rf_randomcv.best_params_['min_samples_split'] + 2],
    'n_estimators': [rf_randomcv.best_params_['n_estimators'] - 200, rf_randomcv.best_params_['n_estimators'] - 100, 
                     rf_randomcv.best_params_['n_estimators'], 
                     rf_randomcv.best_params_['n_estimators'] + 100, rf_randomcv.best_params_['n_estimators'] + 200]
}

print(param_grid)
#### Fit the grid_search to the data
rf=RandomForestClassifier()
grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)
grid_search.best_estimator_
y_pred=best_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: {}".format(classification_report(y_test,y_pred)))

NameError: name 'rf_randomcv' is not defined

# Automated Hyperparameter Tuning

Automated Hyperparameter Tuning can be done by using techniques such as

Bayesian Optimization
Gradient Descent
Evolutionary Algorithms
Bayesian Optimization

Bayesian optimization uses probability to find the minimum of a function.
The final aim is to find the input value to a function which can gives us the lowest possible output value.
It usually performs better than random,grid and manual search providing better performance in the testing phase and reduced optimization time. In Hyperopt, Bayesian Optimization can be implemented giving 3 three main parameters to the function fmin.

Objective Function = defines the loss function to minimize.

Domain Space = defines the range of input values to test (in Bayesian Optimization this space creates a probability distribution for each of the used Hyperparameters).

Optimization Algorithm = defines the search algorithm to use to select the best input values to use in each new iteration.

In [46]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [48]:
space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.quniform('max_depth', 10, 1200, 10),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.choice('n_estimators', [10, 50, 300, 750, 1200,1300,1500])
    }

In [51]:
def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = space['max_depth'],
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = space['n_estimators'], 
                                 )
    
    accuracy = cross_val_score(model, X_train, y_train, cv = 5).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK }

In [ ]:
from sklearn.model_selection import cross_val_score
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 80,
            trials= trials)
best

In [ ]:
crit = {0: 'entropy', 1: 'gini'}
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 10, 1: 50, 2: 300, 3: 750, 4: 1200,5:1300,6:1500}


print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

In [ ]:
best['min_samples_leaf']
trainedforest = RandomForestClassifier(criterion = crit[best['criterion']], max_depth = best['max_depth'], 
                                       max_features = feat[best['max_features']], 
                                       min_samples_leaf = best['min_samples_leaf'], 
                                       min_samples_split = best['min_samples_split'], 
                                       n_estimators = est[best['n_estimators']]).fit(X_train,y_train)
predictionforest = trainedforest.predict(X_test)
print(confusion_matrix(y_test,predictionforest))
print(accuracy_score(y_test,predictionforest))
print(classification_report(y_test,predictionforest))
acc5 = accuracy_score(y_test,predictionforest)